# **Bronze to Silver Transformation — DimProduct (Categories, Subcategories, Products)**

## Introduction
This notebook transforms AdventureWorks Product data from the Bronze layer into curated Silver tables:
- DimProductCategory
- DimProductSubcategory
- DimProduct

The notebook:
- Loads three Bronze CSV sources
- Cleans and standardizes category & subcategory lookups
- Cleans product data
- Adds derived fields including ProductSKUPrefix
- Writes all outputs as managed Delta tables into Silver layer

In [8]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

# Load Bronze Data
df_ProCat_Bronze = (
    spark.read.format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .load("abfss://MonoWS@onelake.dfs.fabric.microsoft.com/MonoLH_Bronze.Lakehouse/Files/Raw/AdventureWorks_Product_Categories/AdventureWorks_Product_Categories.csv")
)

df_ProSubcat_Bronze = (
    spark.read.format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .load("abfss://MonoWS@onelake.dfs.fabric.microsoft.com/MonoLH_Bronze.Lakehouse/Files/Raw/AdventureWorks_Product_Subcategories/AdventureWorks_Product_Subcategories.csv")
)

df_Products_Bronze = (
    spark.read.format("csv")
    .option("header", "true")
    .option("inferSchema", "true")
    .load("abfss://MonoWS@onelake.dfs.fabric.microsoft.com/MonoLH_Bronze.Lakehouse/Files/Raw/AdventureWorks_Products/AdventureWorks_Products.csv")
)

StatementMeta(, 2f8c9042-3887-456f-98a3-b56710e15a62, 10, Finished, Available, Finished)

In [9]:
# Preview dtype and data
df_ProCat_Bronze.printSchema()
df_ProCat_Bronze.show()

df_ProSubcat_Bronze.printSchema()
df_ProSubcat_Bronze.show()

df_Products_Bronze.printSchema()
df_Products_Bronze.show()

StatementMeta(, 2f8c9042-3887-456f-98a3-b56710e15a62, 11, Finished, Available, Finished)

root
 |-- ProductCategoryKey: integer (nullable = true)
 |-- CategoryName: string (nullable = true)

+------------------+------------+
|ProductCategoryKey|CategoryName|
+------------------+------------+
|                 1|       Bikes|
|                 2|  Components|
|                 3|    Clothing|
|                 4| Accessories|
+------------------+------------+

root
 |-- ProductSubcategoryKey: integer (nullable = true)
 |-- SubcategoryName: string (nullable = true)
 |-- ProductCategoryKey: integer (nullable = true)

+---------------------+---------------+------------------+
|ProductSubcategoryKey|SubcategoryName|ProductCategoryKey|
+---------------------+---------------+------------------+
|                    1| Mountain Bikes|                 1|
|                    2|     Road Bikes|                 1|
|                    3|  Touring Bikes|                 1|
|                    4|     Handlebars|                 2|
|                    5|Bottom Brackets|                

In [10]:
cols_to_check = ["ProductSKU", "ProductSize", "ProductStyle"]

for c in cols_to_check:
    print(f"\n===== {c} =====")
    df_Products_Bronze.groupBy(c).count().show()

StatementMeta(, 2f8c9042-3887-456f-98a3-b56710e15a62, 12, Finished, Available, Finished)


===== ProductSKU =====
+----------+-----+
|ProductSKU|count|
+----------+-----+
|BK-R50B-48|    1|
|BK-R68R-60|    1|
|   HB-T928|    1|
|FR-M63B-44|    1|
| TG-W091-L|    1|
|FR-M94B-42|    1|
| SH-M897-S|    1|
| SO-B909-L|    1|
|FR-R38B-52|    1|
|BK-T18Y-54|    1|
|BK-R79Y-42|    1|
| SH-W890-S|    1|
|   BC-M005|    1|
|   CL-9009|    1|
|FR-R38R-60|    1|
|   FW-R623|    1|
|BK-M82S-38|    1|
|BK-T18U-62|    1|
|   TI-M602|    1|
|FR-T67Y-54|    1|
+----------+-----+
only showing top 20 rows


===== ProductSize =====
+-----------+-----+
|ProductSize|count|
+-----------+-----+
|         54|    9|
|         XL|    3|
|         42|   15|
|         52|   16|
|          0|   84|
|         70|    1|
|         46|   11|
|          M|   11|
|          L|   11|
|         60|   11|
|         38|   12|
|         40|   11|
|         44|   29|
|         58|   13|
|          S|    9|
|         48|   25|
|         56|    2|
|         62|   11|
|         50|    9|
+-----------+-----+


===== P

In [11]:
# Transform Customer Data
from pyspark.sql.functions import substring, col, trim, to_date

df_Products_Silver = (
    df_Products_Bronze
        # ProductSKU prefix
        .withColumn("ProductSKUPrefix", substring(col("ProductSKU"), 1, 2))

        # Remove duplicated products
        .dropDuplicates(["ProductKey"])
)

df_Products_Silver.show()
df_Products_Silver.printSchema()

StatementMeta(, 2f8c9042-3887-456f-98a3-b56710e15a62, 13, Finished, Available, Finished)

+----------+---------------------+----------+--------------------+--------------------+--------------------+------------+-----------+------------+-----------+------------+----------------+
|ProductKey|ProductSubcategoryKey|ProductSKU|         ProductName|           ModelName|  ProductDescription|ProductColor|ProductSize|ProductStyle|ProductCost|ProductPrice|ProductSKUPrefix|
+----------+---------------------+----------+--------------------+--------------------+--------------------+------------+-----------+------------+-----------+------------+----------------+
|       214|                   31| HL-U509-R|Sport-100 Helmet,...|           Sport-100|Universal fit, we...|         Red|          0|           0|    13.0863|       34.99|              HL|
|       215|                   31|   HL-U509|Sport-100 Helmet,...|           Sport-100|Universal fit, we...|       Black|          0|           0|    12.0278|     33.6442|              HL|
|       218|                   23| SO-B909-M|Mountain B

In [12]:
df_Products_Silver.select("ProductSKU", "ProductSKUPrefix").show(20, truncate=False)

StatementMeta(, 2f8c9042-3887-456f-98a3-b56710e15a62, 14, Finished, Available, Finished)

+----------+----------------+
|ProductSKU|ProductSKUPrefix|
+----------+----------------+
|HL-U509-R |HL              |
|HL-U509   |HL              |
|SO-B909-M |SO              |
|SO-B909-L |SO              |
|HL-U509-B |HL              |
|CA-1098   |CA              |
|LJ-0192-S |LJ              |
|LJ-0192-M |LJ              |
|LJ-0192-L |LJ              |
|LJ-0192-X |LJ              |
|FR-R92R-62|FR              |
|FR-R92R-44|FR              |
|FR-R92R-48|FR              |
|FR-R92R-52|FR              |
|FR-R92R-56|FR              |
|FR-R38B-58|FR              |
|FR-R38B-60|FR              |
|FR-R38B-62|FR              |
|FR-R38R-44|FR              |
|FR-R38R-48|FR              |
+----------+----------------+
only showing top 20 rows



In [13]:
# Write DimProductCategories Table to Silver Layer (Delta Format)
df_ProCat_Bronze.write \
    .mode("append") \
    .format("delta") \
    .saveAsTable("DimProductCategories")

# Write DimProductSubategories Table to Silver Layer (Delta Format)
df_ProSubcat_Bronze.write\
    .mode("append")\
    .format("delta")\
    .saveAsTable("DimProductSubcategories")

# Write DimProduct Table to Silver Layer (Delta Format)
df_Products_Silver.write\
    .mode("append")\
    .format("delta")\
    .saveAsTable("DimProducts")

StatementMeta(, 2f8c9042-3887-456f-98a3-b56710e15a62, 15, Finished, Available, Finished)

In [14]:
# Verify if Silver write Succeeded
df_dimproduct_categories_silver_check = spark.read.table("DimProductCategories")
df_dimproduct_subcategories_silver_check = spark.read.table("DimProductSubcategories")
df_dimproducts_silver_check = spark.read.table("DimProducts")

df_dimproduct_categories_silver_check.show(5)
df_dimproduct_categories_silver_check.printSchema()

df_dimproduct_subcategories_silver_check.show(5)
df_dimproduct_subcategories_silver_check.printSchema()

df_dimproducts_silver_check.show(5)
df_dimproducts_silver_check.printSchema()

StatementMeta(, 2f8c9042-3887-456f-98a3-b56710e15a62, 16, Finished, Available, Finished)

+------------------+------------+
|ProductCategoryKey|CategoryName|
+------------------+------------+
|                 1|       Bikes|
|                 2|  Components|
|                 3|    Clothing|
|                 4| Accessories|
|                 1|       Bikes|
+------------------+------------+
only showing top 5 rows

root
 |-- ProductCategoryKey: integer (nullable = true)
 |-- CategoryName: string (nullable = true)

+---------------------+---------------+------------------+
|ProductSubcategoryKey|SubcategoryName|ProductCategoryKey|
+---------------------+---------------+------------------+
|                    1| Mountain Bikes|                 1|
|                    2|     Road Bikes|                 1|
|                    3|  Touring Bikes|                 1|
|                    4|     Handlebars|                 2|
|                    5|Bottom Brackets|                 2|
+---------------------+---------------+------------------+
only showing top 5 rows

root
 |-- Pr